# ⚙️ Concurrency, Race Conditions, and Deadlocks in Distributed Systems
This notebook demonstrates **concurrent events**, **concurrency control**, and **deadlock handling** using Python examples.

You will explore:
1. Race conditions caused by concurrent access to shared resources.
2. How locks (mutexes) solve concurrency issues.
3. How deadlocks arise.
4. Techniques to prevent deadlocks (lock ordering).

By the end, you will understand how concurrency is controlled and how deadlocks can be avoided in practice.

## 🔹 Example 1: Race Condition vs Concurrency Control
In this example, two threads (Deposit & Withdraw) update a shared resource (`balance`).

- Without locks → inconsistent results due to race conditions.
- With locks → correct, consistent results.

In [ ]:

import threading
import time

balance = 1000  # Shared resource (bank account)
lock = threading.Lock()  # Lock for concurrency control

def deposit(amount):
    global balance
    for _ in range(3):
        # Critical section protected by lock
        with lock:
            temp = balance  # read
            print(f"[Deposit] Reading balance: {temp}")
            time.sleep(0.5)  # simulate delay
            balance = temp + amount  # update
            print(f"[Deposit] Updated balance: {balance}")

def withdraw(amount):
    global balance
    for _ in range(3):
        with lock:
            temp = balance
            print(f"[Withdraw] Reading balance: {temp}")
            time.sleep(0.5)
            if temp >= amount:
                balance = temp - amount
                print(f"[Withdraw] Updated balance: {balance}")
            else:
                print("[Withdraw] Insufficient funds!")

# Run deposit and withdraw concurrently
t1 = threading.Thread(target=deposit, args=(100,))
t2 = threading.Thread(target=withdraw, args=(50,))

t1.start()
t2.start()
t1.join()
t2.join()

print(f"Final Balance: {balance}")


## 🔹 Example 2: Deadlock Scenario
Two threads try to acquire two locks in **different orders**.

- Task1 locks `lock1` then waits for `lock2`.
- Task2 locks `lock2` then waits for `lock1`.

⚠️ This causes a **deadlock** — both wait forever.

In [ ]:

lock1 = threading.Lock()
lock2 = threading.Lock()

def task1():
    with lock1:
        print("Task1 acquired Lock1")
        time.sleep(1)
        with lock2:
            print("Task1 acquired Lock2")

def task2():
    with lock2:
        print("Task2 acquired Lock2")
        time.sleep(1)
        with lock1:
            print("Task2 acquired Lock1")

t1 = threading.Thread(target=task1)
t2 = threading.Thread(target=task2)

t1.start()
t2.start()
t1.join()
t2.join()


## 🔹 Example 3: Deadlock Prevention with Lock Ordering
To prevent deadlocks, all threads acquire locks in the **same order**.

- Both Task1 and Task2 acquire `lock1` first, then `lock2`.
- No circular waiting → **no deadlock**.

In [ ]:

def task1_fixed():
    with lock1:
        with lock2:
            print("Task1 safely acquired Lock1 & Lock2")

def task2_fixed():
    with lock1:
        with lock2:
            print("Task2 safely acquired Lock1 & Lock2")

t1 = threading.Thread(target=task1_fixed)
t2 = threading.Thread(target=task2_fixed)

t1.start()
t2.start()
t1.join()
t2.join()


## 🎯 Student Exercises
1. Modify the deposit/withdraw code to run **without locks** and observe inconsistent balances.
2. Run the deadlock code → observe program hanging.
3. Apply lock ordering to fix the deadlock.
4. Extend the deadlock simulation with **3 locks** and show how circular wait arises.
5. Discuss: How would concurrency control differ in a **distributed system** (e.g., token-based mutual exclusion)?